#DATATHON

Luene Pizzi Mantovani

Objetivo: Desenvolver um modelo de Machine Learning que prevê a probabilidade de um candidato ser contratado para uma vaga dadas características pessoais e profissionais.

### ETAPA 1 - PRÉ-PROCESSAMENTO DOS DADOS

In [1407]:
# IMPORTAR BIBLIOTECAS NECESSÁRIAS
import pandas as pd
import numpy as np
import re
import unicodedata
from difflib import SequenceMatcher
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
from sklearn.ensemble import RandomForestClassifier
import shap
import pickle
import warnings
from dateutil.relativedelta import relativedelta

In [1408]:
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=pd.errors.SettingWithCopyWarning)

In [1409]:
# LER ARQUIVOS JSON COM DADOS PARA MODELAGEM
# BASE APPLICANTS - DADOS DOS CANDIDATOS QUE SE INSCREVERAM PARA AS VAGAS
df_applicants = pd.read_json('/content/applicants.json', orient='index')

infos_basicas = pd.json_normalize(df_applicants['infos_basicas'])
informacoes_pessoais = pd.json_normalize(df_applicants['informacoes_pessoais'])
informacoes_profissionais = pd.json_normalize(df_applicants['informacoes_profissionais'])
formacao_e_idiomas = pd.json_normalize(df_applicants['formacao_e_idiomas'])
cargo_atual = pd.json_normalize(df_applicants['cargo_atual'])

df_applicants = pd.concat([infos_basicas, informacoes_pessoais, informacoes_profissionais, formacao_e_idiomas, cargo_atual], axis=1)

df_applicants.head()

,telefone_recado,telefone,objetivo_profissional,data_criacao,inserido_por,email,local,sabendo_de_nos_por,data_atualizacao,codigo_profissional,...,id_ibrati,email_corporativo,cargo_atual,projeto_atual,cliente,unidade,data_admissao,data_ultima_promocao,nome_superior_imediato,email_superior_imediato
0,,(11) 97048-2708,,10-11-2021 07:29:49,Luna Correia,carolina_aparecida@gmail.com,,,10-11-2021 07:29:49,31000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,,(11) 93723-4396,Analista Administrativo,10-11-2021 08:56:16,Laura Pacheco,eduardo_rios@hotmail.com,"São Paulo, São Paulo",Outros,11-11-2021 11:10:31,31001,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,,(11) 92399-9824,Administrativo | Financeiro,10-11-2021 09:01:00,Laura Pacheco,pedro_henrique_carvalho@gmail.com,"São Paulo, São Paulo",Anúncio,10-11-2021 11:42:36,31002,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,,(11) 98100-1727,Área administrativa,10-11-2021 09:08:13,Laura Pacheco,thiago_barbosa@hotmail.com,"São Paulo, São Paulo",Site de Empregos,10-11-2021 16:04:51,31003,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,,(11) 92517-2678,,10-11-2021 09:18:46,Maria Clara Pires,diogo_das_neves@hotmail.com,,,10-11-2021 09:22:03,31004,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1410]:
df_applicants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42482 entries, 0 to 42481
Data columns (total 55 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   telefone_recado              42482 non-null  object
 1   telefone                     42482 non-null  object
 2   objetivo_profissional        42482 non-null  object
 3   data_criacao                 42482 non-null  object
 4   inserido_por                 42482 non-null  object
 5   email                        42482 non-null  object
 6   local                        42482 non-null  object
 7   sabendo_de_nos_por           42482 non-null  object
 8   data_atualizacao             42482 non-null  object
 9   codigo_profissional          42482 non-null  object
 10  nome                         42482 non-null  object
 11  data_aceite                  42482 non-null  object
 12  nome                         42482 non-null  object
 13  cpf                          42

In [1411]:
# SELECIONAR  APENAS COLUNAS DE INTERESSE
df_applicants_2 = df_applicants[['codigo_profissional', 'local', 'objetivo_profissional',
                                 'pcd', 'titulo_profissional', 'area_atuacao',
                                 'conhecimentos_tecnicos', 'certificacoes', 'remuneracao',
                                 'nivel_profissional', 'nivel_academico',
                                 'nivel_ingles', 'nivel_espanhol', 'cursos', 'ano_conclusao', 'cargo_atual']]

In [1412]:
df_applicants_2.head()

,codigo_profissional,local,objetivo_profissional,pcd,titulo_profissional,area_atuacao,conhecimentos_tecnicos,certificacoes,remuneracao,nivel_profissional,nivel_academico,nivel_ingles,nivel_espanhol,cursos,ano_conclusao,cargo_atual
0,31000,,,,,,,,,,,,,NaN,NaN,NaN
1,31001,"São Paulo, São Paulo",Analista Administrativo,Não,Analista Administrativo,Administrativa,,,1900,,Ensino Superior Incompleto,Nenhum,Nenhum,NaN,NaN,NaN
2,31002,"São Paulo, São Paulo",Administrativo | Financeiro,Não,Administrativo | Financeiro,Administrativa,,"MS [77-418] MOS: Microsoft Office Word 2013, M...","2.500,00",,Ensino Superior Completo,Intermediário,Básico,Administração de Empresas,2012,NaN
3,31003,"São Paulo, São Paulo",Área administrativa,Não,Área administrativa,Administrativa,,,"1100,00",,Ensino Superior Incompleto,Nenhum,Nenhum,NaN,NaN,NaN
4,31004,,,,,,,,,,,,,NaN,NaN,NaN


In [1413]:
df_applicants_2.dtypes

,0
codigo_profissional,object
local,object
objetivo_profissional,object
pcd,object
titulo_profissional,object
area_atuacao,object
conhecimentos_tecnicos,object
certificacoes,object
remuneracao,object
nivel_profissional,object


In [1414]:
df_applicants_2[['cidade', 'estado']] = df_applicants_2['local'].str.split(r'\s*,\s*', n=1, expand=True)

In [1415]:
pat = r'((?:\d{1,3}(?:[.\s]\d{3})+|\d+)(?:[.,]\d+)?)'

df_applicants_2['remuneracao_num'] = (
    df_applicants_2['remuneracao']
      .astype('string')
      .str.extract(pat, expand=False)           # agora tem grupo de captura ✅
      .str.replace(r'[.\s]', '', regex=True)    # remove milhar (ponto/espaço)
      .str.replace(',', '.', regex=False)       # vírgula -> ponto
      .astype('float64')
)


In [1416]:
pd.options.display.float_format = '{:.2f}'.format
display(df_applicants_2['remuneracao_num'].describe())

,remuneracao_num
count,19976.00
mean,3772.76
std,76477.41
min,0.00
25%,0.00
50%,0.00
75%,2178.95
max,10000000.00


In [1417]:
# BASE PROSPECTS - DADOS DOS CANDIDATOS QUE FORAM SELECIONADOS PARA AS VAGAS
df_prospects = pd.read_json('/content/prospects.json', orient='index')

base_vaga = df_prospects.drop(columns=['prospects'])

base_explodida = df_prospects[['prospects']].explode('prospects')

base_explodida_valid = base_explodida.dropna(subset=['prospects'])

base_candidatos = pd.DataFrame.from_records(base_explodida_valid['prospects'].array,
                                 index=base_explodida_valid.index)

df_prospects = base_vaga.join(base_candidatos, how='right')
df_prospects.index.name = 'id_vaga'

df_prospects = df_prospects.reset_index()

In [1418]:
df_prospects.head()

,id_vaga,titulo,modalidade,nome,codigo,situacao_candidado,data_candidatura,ultima_atualizacao,comentario,recrutador
0,4530,CONSULTOR CONTROL M,,José Vieira,25632,Encaminhado ao Requisitante,25-03-2021,25-03-2021,"Encaminhado para - PJ R$ 72,00/hora",Ana Lívia Moreira
1,4530,CONSULTOR CONTROL M,,Srta. Isabela Cavalcante,25529,Encaminhado ao Requisitante,22-03-2021,23-03-2021,"encaminhado para - R$ 6.000,00 – CLT Full , n...",Ana Lívia Moreira
2,4531,2021-2607395-PeopleSoft Application Engine-Dom...,,Sra. Yasmin Fernandes,25364,Contratado pela Decision,17-03-2021,12-04-2021,Data de Inicio: 12/04/2021,Juliana Cassiano
3,4531,2021-2607395-PeopleSoft Application Engine-Dom...,,Alexia Barbosa,25360,Encaminhado ao Requisitante,17-03-2021,17-03-2021,,Juliana Cassiano
4,4533,2021-2605708-Microfocus Application Life Cycle...,,Arthur Almeida,26338,Contratado pela Decision,29-04-2021,18-05-2021,,Stella Vieira


In [1419]:
df_prospects.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53759 entries, 0 to 53758
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id_vaga             53759 non-null  int64 
 1   titulo              53759 non-null  object
 2   modalidade          53759 non-null  object
 3   nome                53759 non-null  object
 4   codigo              53759 non-null  object
 5   situacao_candidado  53759 non-null  object
 6   data_candidatura    53759 non-null  object
 7   ultima_atualizacao  53759 non-null  object
 8   comentario          53759 non-null  object
 9   recrutador          53759 non-null  object
dtypes: int64(1), object(9)
memory usage: 4.1+ MB


In [1420]:
df_prospects.dtypes

,0
id_vaga,int64
titulo,object
modalidade,object
nome,object
codigo,object
situacao_candidado,object
data_candidatura,object
ultima_atualizacao,object
comentario,object
recrutador,object


In [1421]:
df_prospects['modalidade'].value_counts(dropna=False)

,count
modalidade,
,52076
Cooperado,590
PJ,540
CLT,406
Hunting,77
CLT - Estratégico,70


In [1422]:
df_prospects['situacao_candidado'].value_counts(dropna=False)

,count
situacao_candidado,
Prospect,20021
Encaminhado ao Requisitante,16122
Inscrito,3980
Não Aprovado pelo Cliente,3492
Contratado pela Decision,2758
Desistiu,2349
Não Aprovado pelo RH,1765
Não Aprovado pelo Requisitante,765
Entrevista Técnica,579


In [1423]:
#SELECIONAR COLUNAS DE INTERESSE PARA MODELAGEM
df_prospects_2 = df_prospects[['id_vaga', 'titulo', 'modalidade', 'nome', 'codigo', 'situacao_candidado']]

In [1424]:
df_prospects_2.head()

,id_vaga,titulo,modalidade,nome,codigo,situacao_candidado
0,4530,CONSULTOR CONTROL M,,José Vieira,25632,Encaminhado ao Requisitante
1,4530,CONSULTOR CONTROL M,,Srta. Isabela Cavalcante,25529,Encaminhado ao Requisitante
2,4531,2021-2607395-PeopleSoft Application Engine-Dom...,,Sra. Yasmin Fernandes,25364,Contratado pela Decision
3,4531,2021-2607395-PeopleSoft Application Engine-Dom...,,Alexia Barbosa,25360,Encaminhado ao Requisitante
4,4533,2021-2605708-Microfocus Application Life Cycle...,,Arthur Almeida,26338,Contratado pela Decision


In [1425]:
df_prospects_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53759 entries, 0 to 53758
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id_vaga             53759 non-null  int64 
 1   titulo              53759 non-null  object
 2   modalidade          53759 non-null  object
 3   nome                53759 non-null  object
 4   codigo              53759 non-null  object
 5   situacao_candidado  53759 non-null  object
dtypes: int64(1), object(5)
memory usage: 2.5+ MB


In [1426]:
# BASE VAGAS - INFORMAÇÕES SOBRE AS VAGAS DISPONIBILIZADAS E TRABALHADAS PELA DECISION
df_vagas = pd.read_json('/content/vagas.json', orient='index')

df_info_basicas = pd.json_normalize(df_vagas['informacoes_basicas']).set_index(df_vagas.index)
df_perfil_vaga  = pd.json_normalize(df_vagas['perfil_vaga']).set_index(df_vagas.index)
df_beneficios   = pd.json_normalize(df_vagas['beneficios']).set_index(df_vagas.index)

df_vagas = pd.concat([df_info_basicas, df_perfil_vaga, df_beneficios], axis=1)

df_vagas = df_vagas.reset_index().rename(columns={'index': 'id_vaga'})

df_vagas.head()

,id_vaga,data_requicisao,limite_esperado_para_contratacao,titulo_vaga,vaga_sap,cliente,solicitante_cliente,empresa_divisao,requisitante,analista_responsavel,...,areas_atuacao,principais_atividades,competencia_tecnicas_e_comportamentais,demais_observacoes,viagens_requeridas,equipamentos_necessarios,habilidades_comportamentais_necessarias,valor_venda,valor_compra_1,valor_compra_2
0,5185,04-05-2021,00-00-0000,Operation Lead -,Não,"Morris, Moran and Dodson",Dra. Catarina Marques,Decision São Paulo,Maria Laura Nogueira,Srta. Bella Ferreira,...,TI - Sistemas e Ferramentas-,Operations Lead\n\nRoles & Responsibilities:\n...,Required Skills:\n• Prior experience in Cloud ...,100% Remoto Período – entre 5 – 6 meses,,Nenhum -,NaN,-,R$,
1,5184,04-05-2021,00-00-0000,Consultor PP/QM Sênior,Não,"Morris, Moran and Dodson",Dra. Catarina Marques,Decision São Paulo,Maria Laura Nogueira,Yasmin da Rosa,...,TI - Desenvolvimento/Programação-,Consultor PP/QM Sr.\n\n• Consultor PP/QM Sênio...,• Consultor PP/QM Sênior com experiencia em pr...,• Início: Imediato • Fim: Jan/22,,Nenhum -,NaN,-,R$,
2,5183,04-05-2021,00-00-0000,ANALISTA PL/JR C/ SQL,Não,"Morris, Moran and Dodson",Dra. Catarina Marques,Decision São Paulo,Maria Laura Nogueira,Ana Albuquerque,...,TI - Sistemas e Ferramentas-,Descrição – Atividades:\n\no Monitoramento das...,Requisitos mandatórios:\n\no Conhecimentos Téc...,Localização: Remoto Perfil: Analista Pleno ou ...,,Nenhum -,NaN,-,R$,
3,5182,04-05-2021,18-05-2021,Technical Architect - 11894809,Não,Nelson-Page,Dr. Raul Monteiro,Decision São Paulo,Cecília Freitas,Clara Rios,...,TI - Projetos-,Descrição/Comentário: Architecture Frameworks ...,Descrição/Comentário: Architecture Frameworks ...,Budgeted Rate - indicate currency and type (ho...,Não,Notebook padrão -,NaN,- p/ mês (168h),fechado,
4,5181,04-05-2021,00-00-0000,Consultor SAP AUTHORIZATION (BCA) -Pleno / Sênior,Não,Mann and Sons,Cauê Fogaça,Decision São Paulo,Maria Laura Nogueira,Srta. Bella Ferreira,...,TI - SAP-,Experiência como Consultor SAP AUTHORIZATION (...,Experiência como Consultor SAP AUTHORIZATION (...,contratação CLT full pela Decision locação rem...,Sim,Nenhum -,NaN,-,R$,


In [1427]:
df_vagas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14081 entries, 0 to 14080
Data columns (total 45 columns):
 #   Column                                   Non-Null Count  Dtype 
---  ------                                   --------------  ----- 
 0   id_vaga                                  14081 non-null  int64 
 1   data_requicisao                          14081 non-null  object
 2   limite_esperado_para_contratacao         14081 non-null  object
 3   titulo_vaga                              14081 non-null  object
 4   vaga_sap                                 14081 non-null  object
 5   cliente                                  14081 non-null  object
 6   solicitante_cliente                      14081 non-null  object
 7   empresa_divisao                          14081 non-null  object
 8   requisitante                             14081 non-null  object
 9   analista_responsavel                     14081 non-null  object
 10  tipo_contratacao                         14081 non-null  o

In [1428]:
# SELECIONAR  APENAS COLUNAS DE INTERESSE
df_vagas_2 = df_vagas[['id_vaga', 'titulo_vaga', 'vaga_sap', 'tipo_contratacao', 'pais', 'estado', 'cidade', 'regiao',
                       'vaga_especifica_para_pcd', 'nivel profissional', 'nivel_academico',
                       'nivel_ingles', 'nivel_espanhol', 'areas_atuacao', 'principais_atividades',
                       'competencia_tecnicas_e_comportamentais', 'demais_observacoes', 'habilidades_comportamentais_necessarias']]

In [1429]:
df_vagas_2.head()

,id_vaga,titulo_vaga,vaga_sap,tipo_contratacao,pais,estado,cidade,regiao,vaga_especifica_para_pcd,nivel profissional,nivel_academico,nivel_ingles,nivel_espanhol,areas_atuacao,principais_atividades,competencia_tecnicas_e_comportamentais,demais_observacoes,habilidades_comportamentais_necessarias
0,5185,Operation Lead -,Não,CLT Full,Brasil,São Paulo,São Paulo,,Não,Sênior,Ensino Superior Completo,Avançado,Fluente,TI - Sistemas e Ferramentas-,Operations Lead\n\nRoles & Responsibilities:\n...,Required Skills:\n• Prior experience in Cloud ...,100% Remoto Período – entre 5 – 6 meses,NaN
1,5184,Consultor PP/QM Sênior,Não,CLT Full,Brasil,São Paulo,São Paulo,,Não,Sênior,Ensino Superior Completo,Fluente,Nenhum,TI - Desenvolvimento/Programação-,Consultor PP/QM Sr.\n\n• Consultor PP/QM Sênio...,• Consultor PP/QM Sênior com experiencia em pr...,• Início: Imediato • Fim: Jan/22,NaN
2,5183,ANALISTA PL/JR C/ SQL,Não,CLT Full,Brasil,São Paulo,São Paulo,,Não,Analista,Ensino Superior Completo,Nenhum,Intermediário,TI - Sistemas e Ferramentas-,Descrição – Atividades:\n\no Monitoramento das...,Requisitos mandatórios:\n\no Conhecimentos Téc...,Localização: Remoto Perfil: Analista Pleno ou ...,NaN
3,5182,Technical Architect - 11894809,Não,"PJ/Autônomo, CLT Full",Brasil,São Paulo,São Paulo,,Não,Analista,Ensino Superior Completo,Básico,Básico,TI - Projetos-,Descrição/Comentário: Architecture Frameworks ...,Descrição/Comentário: Architecture Frameworks ...,Budgeted Rate - indicate currency and type (ho...,NaN
4,5181,Consultor SAP AUTHORIZATION (BCA) -Pleno / Sênior,Não,CLT Full,Brasil,São Paulo,São Paulo,,Não,Sênior,Ensino Superior Completo,Intermediário,Nenhum,TI - SAP-,Experiência como Consultor SAP AUTHORIZATION (...,Experiência como Consultor SAP AUTHORIZATION (...,contratação CLT full pela Decision locação rem...,NaN


In [1430]:
df_vagas_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14081 entries, 0 to 14080
Data columns (total 18 columns):
 #   Column                                   Non-Null Count  Dtype 
---  ------                                   --------------  ----- 
 0   id_vaga                                  14081 non-null  int64 
 1   titulo_vaga                              14081 non-null  object
 2   vaga_sap                                 14081 non-null  object
 3   tipo_contratacao                         14081 non-null  object
 4   pais                                     14081 non-null  object
 5   estado                                   14081 non-null  object
 6   cidade                                   14081 non-null  object
 7   regiao                                   14081 non-null  object
 8   vaga_especifica_para_pcd                 14081 non-null  object
 9   nivel profissional                       14081 non-null  object
 10  nivel_academico                          14081 non-null  o

In [1431]:
df_vagas_2.dtypes

,0
id_vaga,int64
titulo_vaga,object
vaga_sap,object
tipo_contratacao,object
pais,object
estado,object
cidade,object
regiao,object
vaga_especifica_para_pcd,object
nivel profissional,object


In [1432]:
#UNIFICAR AS TRÊS BASES
df_consolidado = df_vagas_2.merge(df_prospects_2, on='id_vaga', how='left')

In [1433]:
df_consolidado.head()

,id_vaga,titulo_vaga,vaga_sap,tipo_contratacao,pais,estado,cidade,regiao,vaga_especifica_para_pcd,nivel profissional,...,areas_atuacao,principais_atividades,competencia_tecnicas_e_comportamentais,demais_observacoes,habilidades_comportamentais_necessarias,titulo,modalidade,nome,codigo,situacao_candidado
0,5185,Operation Lead -,Não,CLT Full,Brasil,São Paulo,São Paulo,,Não,Sênior,...,TI - Sistemas e Ferramentas-,Operations Lead\n\nRoles & Responsibilities:\n...,Required Skills:\n• Prior experience in Cloud ...,100% Remoto Período – entre 5 – 6 meses,NaN,Operation Lead -,,Dante Nascimento,11010,Encaminhado ao Requisitante
1,5184,Consultor PP/QM Sênior,Não,CLT Full,Brasil,São Paulo,São Paulo,,Não,Sênior,...,TI - Desenvolvimento/Programação-,Consultor PP/QM Sr.\n\n• Consultor PP/QM Sênio...,• Consultor PP/QM Sênior com experiencia em pr...,• Início: Imediato • Fim: Jan/22,NaN,Consultor PP/QM Sênior,,Samuel Costa,26770,Prospect
2,5184,Consultor PP/QM Sênior,Não,CLT Full,Brasil,São Paulo,São Paulo,,Não,Sênior,...,TI - Desenvolvimento/Programação-,Consultor PP/QM Sr.\n\n• Consultor PP/QM Sênio...,• Consultor PP/QM Sênior com experiencia em pr...,• Início: Imediato • Fim: Jan/22,NaN,Consultor PP/QM Sênior,,Maria Laura Brito,26759,Prospect
3,5184,Consultor PP/QM Sênior,Não,CLT Full,Brasil,São Paulo,São Paulo,,Não,Sênior,...,TI - Desenvolvimento/Programação-,Consultor PP/QM Sr.\n\n• Consultor PP/QM Sênio...,• Consultor PP/QM Sênior com experiencia em pr...,• Início: Imediato • Fim: Jan/22,NaN,Consultor PP/QM Sênior,,Raul Monteiro,26758,Prospect
4,5184,Consultor PP/QM Sênior,Não,CLT Full,Brasil,São Paulo,São Paulo,,Não,Sênior,...,TI - Desenvolvimento/Programação-,Consultor PP/QM Sr.\n\n• Consultor PP/QM Sênio...,• Consultor PP/QM Sênior com experiencia em pr...,• Início: Imediato • Fim: Jan/22,NaN,Consultor PP/QM Sênior,,José Miguel Cunha,26757,Prospect


In [1434]:
df_final = df_consolidado.merge(df_applicants_2, left_on='codigo', right_on='codigo_profissional', how='left', suffixes=('', '_candidato'))

In [1435]:
df_final.head()

,id_vaga,titulo_vaga,vaga_sap,tipo_contratacao,pais,estado,cidade,regiao,vaga_especifica_para_pcd,nivel profissional,...,nivel_profissional,nivel_academico_candidato,nivel_ingles_candidato,nivel_espanhol_candidato,cursos,ano_conclusao,cargo_atual,cidade_candidato,estado_candidato,remuneracao_num
0,5185,Operation Lead -,Não,CLT Full,Brasil,São Paulo,São Paulo,,Não,Sênior,...,,,,,NaN,NaN,NaN,,None,0.00
1,5184,Consultor PP/QM Sênior,Não,CLT Full,Brasil,São Paulo,São Paulo,,Não,Sênior,...,,,,,NaN,NaN,NaN,,None,NaN
2,5184,Consultor PP/QM Sênior,Não,CLT Full,Brasil,São Paulo,São Paulo,,Não,Sênior,...,,,,,NaN,NaN,NaN,,None,NaN
3,5184,Consultor PP/QM Sênior,Não,CLT Full,Brasil,São Paulo,São Paulo,,Não,Sênior,...,,,,,NaN,NaN,NaN,Minas Gerais,None,NaN
4,5184,Consultor PP/QM Sênior,Não,CLT Full,Brasil,São Paulo,São Paulo,,Não,Sênior,...,,,,,NaN,NaN,NaN,,None,NaN


In [1436]:
df_final.columns

Index(['id_vaga', 'titulo_vaga', 'vaga_sap', 'tipo_contratacao', 'pais',
       'estado', 'cidade', 'regiao', 'vaga_especifica_para_pcd',
       'nivel profissional', 'nivel_academico', 'nivel_ingles',
       'nivel_espanhol', 'areas_atuacao', 'principais_atividades',
       'competencia_tecnicas_e_comportamentais', 'demais_observacoes',
       'habilidades_comportamentais_necessarias', 'titulo', 'modalidade',
       'nome', 'codigo', 'situacao_candidado', 'codigo_profissional', 'local',
       'objetivo_profissional', 'pcd', 'titulo_profissional', 'area_atuacao',
       'conhecimentos_tecnicos', 'certificacoes', 'remuneracao',
       'nivel_profissional', 'nivel_academico_candidato',
       'nivel_ingles_candidato', 'nivel_espanhol_candidato', 'cursos',
       'ano_conclusao', 'cargo_atual', 'cidade_candidato', 'estado_candidato',
       'remuneracao_num'],
      dtype='object')

In [1437]:
#TRATAR DADOS NULOS
df_final['objetivo_profissional'] = (df_final['objetivo_profissional']
                                 .astype('string')
                                 .str.strip()
                                 .replace('', pd.NA)
                                 .fillna('Não Informado'))

In [1438]:
df_final['cidade_candidato'] = (df_final['cidade_candidato']
                                 .astype('string')
                                 .str.strip()
                                 .replace('', pd.NA)
                                 .fillna('Não Informado'))

In [1439]:
df_final['estado_candidato'] = (df_final['estado_candidato']
                                 .astype('string')
                                 .str.strip()
                                 .replace('', pd.NA)
                                 .fillna('Não Informado'))

In [1440]:
df_final['pcd'] = (df_final['pcd']
                                 .astype('string')
                                 .str.strip()
                                 .replace('', pd.NA)
                                 .fillna('Não Informado'))

In [1441]:
df_final['titulo_profissional'] = (df_final['titulo_profissional']
                                 .astype('string')
                                 .str.strip()
                                 .replace('', pd.NA)
                                 .fillna('Não Informado'))

In [1442]:
df_final['area_atuacao'] = (df_final['area_atuacao']
                                 .astype('string')
                                 .str.strip()
                                 .replace('', pd.NA)
                                 .fillna('Não Informado'))

In [1443]:
df_final['conhecimentos_tecnicos'] = (df_final['conhecimentos_tecnicos']
                                 .astype('string')
                                 .str.strip()
                                 .replace('', pd.NA)
                                 .fillna('Não Informado'))

In [1444]:
df_final['certificacoes'] = (df_final['certificacoes']
                                 .astype('string')
                                 .str.strip()
                                 .replace('', pd.NA)
                                 .fillna('Não Informado'))

In [1445]:
df_final['nivel_profissional'] = (df_final['nivel_profissional']
                                 .astype('string')
                                 .str.strip()
                                 .replace('', pd.NA)
                                 .fillna('Não Informado'))

In [1446]:
df_final['nivel_academico_candidato'] = (df_final['nivel_academico_candidato']
                                 .astype('string')
                                 .str.strip()
                                 .replace('', pd.NA)
                                 .fillna('Não Informado'))

In [1447]:
df_final['nivel_ingles_candidato'] = (df_final['nivel_ingles_candidato']
                                 .astype('string')
                                 .str.strip()
                                 .replace('', pd.NA)
                                 .fillna('Não Informado'))

In [1448]:
df_final['nivel_espanhol_candidato'] = (df_final['nivel_espanhol_candidato']
                                 .astype('string')
                                 .str.strip()
                                 .replace('', pd.NA)
                                 .fillna('Não Informado'))

In [1449]:
df_final['cursos'] = (df_final['cursos']
                                 .astype('string')
                                 .str.strip()
                                 .replace('', pd.NA)
                                 .fillna('Não Informado'))

In [1450]:
df_final['cargo_atual'] = (df_final['cargo_atual']
                                 .astype('string')
                                 .str.strip()
                                 .replace('', pd.NA)
                                 .fillna('Não Informado'))

In [1451]:
df_final['modalidade'] = (df_final['modalidade']
                                 .astype('string')
                                 .str.strip()
                                 .replace('', pd.NA)
                                 .fillna('Não Informado'))

In [1452]:
df_final['tipo_contratacao'] = (df_final['tipo_contratacao']
                                 .astype('string')
                                 .str.strip()
                                 .replace('', pd.NA)
                                 .fillna('Não Informado'))

In [1453]:
df_final['estado'] = (df_final['estado']
                                 .astype('string')
                                 .str.strip()
                                 .replace('', pd.NA)
                                 .fillna('Não Informado'))

In [1454]:
df_final['cidade'] = (df_final['cidade']
                                 .astype('string')
                                 .str.strip()
                                 .replace('', pd.NA)
                                 .fillna('Não Informado'))

In [1455]:
df_final['regiao'] = (df_final['regiao']
                                 .astype('string')
                                 .str.strip()
                                 .replace('', pd.NA)
                                 .fillna('Não Informado'))

In [1456]:
df_final['vaga_especifica_para_pcd'] = (df_final['vaga_especifica_para_pcd']
                                 .astype('string')
                                 .str.strip()
                                 .replace('', pd.NA)
                                 .fillna('Não Informado'))

In [1457]:
df_final['nivel_espanhol'] = (df_final['nivel_espanhol']
                                 .astype('string')
                                 .str.strip()
                                 .replace('', pd.NA)
                                 .fillna('Não Informado'))

In [1458]:
df_final['areas_atuacao'] = (df_final['areas_atuacao']
                                 .astype('string')
                                 .str.strip()
                                 .replace('', pd.NA)
                                 .fillna('Não Informado'))

In [1459]:
df_final['principais_atividades'] = (df_final['principais_atividades']
                                 .astype('string')
                                 .str.strip()
                                 .replace('', pd.NA)
                                 .replace('.', pd.NA)
                                 .fillna('Não Informado'))

In [1460]:
df_final['competencia_tecnicas_e_comportamentais'] = (df_final['competencia_tecnicas_e_comportamentais']
                                 .astype('string')
                                 .str.strip()
                                 .replace('', pd.NA)
                                 .replace('.', pd.NA)
                                 .fillna('Não Informado'))

In [1461]:
df_final['habilidades_comportamentais_necessarias'] = (df_final['habilidades_comportamentais_necessarias']
                                 .astype('string')
                                 .str.strip()
                                 .replace('', pd.NA)
                                 .replace('.', pd.NA)
                                 .fillna('Não Informado'))

In [1462]:
df_final = df_final.drop(columns=['local', 'remuneracao'])

In [1463]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56539 entries, 0 to 56538
Data columns (total 40 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   id_vaga                                  56539 non-null  int64  
 1   titulo_vaga                              56539 non-null  object 
 2   vaga_sap                                 56539 non-null  object 
 3   tipo_contratacao                         56539 non-null  string 
 4   pais                                     56539 non-null  object 
 5   estado                                   56539 non-null  string 
 6   cidade                                   56539 non-null  string 
 7   regiao                                   56539 non-null  string 
 8   vaga_especifica_para_pcd                 56539 non-null  string 
 9   nivel profissional                       56539 non-null  object 
 10  nivel_academico                          56539

In [1464]:
df_final.dropna(subset=['codigo_profissional'], inplace=True)
display(df_final.info())

<class 'pandas.core.frame.DataFrame'>
Index: 45071 entries, 0 to 56538
Data columns (total 40 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   id_vaga                                  45071 non-null  int64  
 1   titulo_vaga                              45071 non-null  object 
 2   vaga_sap                                 45071 non-null  object 
 3   tipo_contratacao                         45071 non-null  string 
 4   pais                                     45071 non-null  object 
 5   estado                                   45071 non-null  string 
 6   cidade                                   45071 non-null  string 
 7   regiao                                   45071 non-null  string 
 8   vaga_especifica_para_pcd                 45071 non-null  string 
 9   nivel profissional                       45071 non-null  object 
 10  nivel_academico                          45071 non-

None

### ETAPA 2 - ANÁLISE EXPLORATÓRIA

In [1465]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45071 entries, 0 to 56538
Data columns (total 40 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   id_vaga                                  45071 non-null  int64  
 1   titulo_vaga                              45071 non-null  object 
 2   vaga_sap                                 45071 non-null  object 
 3   tipo_contratacao                         45071 non-null  string 
 4   pais                                     45071 non-null  object 
 5   estado                                   45071 non-null  string 
 6   cidade                                   45071 non-null  string 
 7   regiao                                   45071 non-null  string 
 8   vaga_especifica_para_pcd                 45071 non-null  string 
 9   nivel profissional                       45071 non-null  object 
 10  nivel_academico                          45071 non-

In [1466]:
df_final.describe()

,id_vaga,remuneracao_num
count,45071.00,15201.00
mean,7181.03,8948.30
std,3679.42,166354.29
min,2.00,0.00
25%,4175.50,0.00
50%,7394.00,2500.00
75%,10300.50,7365.00
max,14220.00,10000000.00


In [1467]:
df_final.describe(include='object')

,titulo_vaga,vaga_sap,pais,nivel profissional,nivel_academico,nivel_ingles,demais_observacoes,titulo,nome,codigo,situacao_candidado,codigo_profissional,ano_conclusao
count,45071,45071,45071,45071,45071,45071,45071,45071,45071,45071,45071,45071,11120
unique,9273,2,1,13,15,6,5424,9273,23174,23457,21,23457,75
top,SAP SD,Não,Brasil,Sênior,Ensino Superior Completo,Básico,,SAP SD,Sra. Maria Helena Castro,833,Prospect,833,0
freq,256,43458,45071,17987,33830,14913,10831,256,73,73,16602,73,1907


### ETAPA 3 - FEATURE ENGINEERING

In [1468]:
df_final['situacao_candidado'].unique()

array(['Encaminhado ao Requisitante', 'Prospect',
       'Não Aprovado pelo Cliente', 'Inscrito', 'Não Aprovado pelo RH',
       'Não Aprovado pelo Requisitante', 'Contratado como Hunting',
       'Desistiu', 'Contratado pela Decision', 'Entrevista Técnica',
       'Em avaliação pelo RH', 'Aprovado', 'Entrevista com Cliente',
       'Documentação PJ', 'Documentação Cooperado',
       'Desistiu da Contratação', 'Recusado', 'Documentação CLT',
       'Sem interesse nesta vaga', 'Proposta Aceita',
       'Encaminhar Proposta'], dtype=object)

In [1469]:
#CRIAR CAMPO PARA INDICAR A SITUAÇÃO FINAL DO CANDIDATO (QUE SERÁ UTILIZADO COMO TARGET PARA O MODELO)
df_final['status_contratacao'] = np.where(
    df_final['situacao_candidado'].isin(['Contratado pela Decision', 'Contratado como Hunting', 'Aprovado']),
    'Contratado',
    'Não Contratado'
)


In [1470]:
df_final['status_contratacao'].value_counts()

,count
status_contratacao,
Não Contratado,42462
Contratado,2609


In [1471]:
#CRIAR COLUNA COM FAIXA DE REMUNERAÇÃO DO CANDIDATO
bins = [0, 1400, 3000, 7000, 15000, float('inf')]
labels = ['a. 0-1400', 'b. 1401-3000', 'c. 3001-7000', 'd. 7001-15000', 'e. +15000']

df_final['faixa_remuneracao'] = pd.cut(df_final['remuneracao_num'], bins=bins, labels=labels, right=True, include_lowest=True)

df_final['faixa_remuneracao'] = df_final['faixa_remuneracao'].cat.add_categories('f. Valor não informado').fillna('f. Valor não informado')


In [1472]:
#CRIAR COLUNA PARA INDICAR SE O CANDIDATO POSSUI ALGUMA CERTIFICAÇÃO OU NÃO
df_final['possui_certificacao'] = np.where(
    (df_final['certificacoes'].notna()) & (df_final['certificacoes'] != 'Não Informado'),
    'Sim',
    'Não'
)

In [1473]:
#AVALIAR TERMOS SEMELHANTES NOS CAMPOS DE ÁREA DE ATUAÇÃO PROFISSIONAL (VAGA X CANDIDATO)


def _strip_accents(s: str) -> str:
    return unicodedata.normalize('NFKD', s).encode('ascii', 'ignore').decode('ascii')

def _norm(s):
    if pd.isna(s):
        return ''
    s = str(s).lower().strip()
    s = _strip_accents(s)
    s = re.sub(r'-+$', '', s)
    s = re.sub(r'\s+', ' ', s).strip()
    return s

# sinônimos
MACRO_MAP = {
    'ti': 'ti',
    'tecnologia da informacao': 'ti',
    'technology': 'ti',
    'information technology': 'ti',
    'rh': 'rh',
    'recursos humanos': 'rh',
    'vendas': 'comercial',
    'comercial': 'comercial',
    'marketing': 'mkt'
}

def _macro_area(s: str) -> str:
    """Pega o trecho antes do primeiro '-' como macro, normaliza e mapeia sinônimos."""
    s = _norm(s)
    parts = re.split(r'\s*-\s*', s, maxsplit=1)
    macro = parts[0].strip()
    macro = MACRO_MAP.get(macro, macro)
    if macro in {'', 'na', 'n/a', 'nao informado', 'nao-informado', 'indefinido'}:
        return ''
    return macro

# sinônimos finos para termos (não macro)
TERM_MAP = {
    'front end': 'frontend', 'front-end': 'frontend', 'front_end': 'frontend',
    'back end': 'backend',  'back-end': 'backend',    'back_end': 'backend',
    'dev ops': 'devops',    'dev-ops': 'devops',
    'cientista de dados': 'data science',
    'analise de dados': 'data analytics', 'analista de dados': 'data analytics',
    'qa': 'quality assurance', 'q a': 'quality assurance',
    'pl/sql': 'plsql', 'pl sql': 'plsql', 'lider' : 'liderança', 'projetos': 'projeto', 'ti': 'tecnologia da informacao',
    'tecnologia da informacao': 'ti', 'liderança': 'lider', 'gerente': 'gerencia', 'gestao': 'lideranca', 'proj': 'projeto',
    'proj': 'projetos'
}

def _to_terms(x):
    """Transforma string/lista em conjunto de termos canônicos (não-macro)."""
    if isinstance(x, list):
        raw = ' | '.join(map(str, x))
    else:
        raw = str(x)
    s = _norm(raw)

    for k, v in TERM_MAP.items():
        s = s.replace(k, v)

    parts = re.split(r'[;,/|]+|\s*-\s*', s)
    parts = [p.strip() for p in parts if p and p.strip()]

    if parts:
        macro = MACRO_MAP.get(parts[0], parts[0])
        parts = parts[1:] if len(parts) > 1 and macro == _macro_area(raw) else parts

    canon = []
    for t in parts:
        t = TERM_MAP.get(t, t)
        canon.append(t)
    canon = [t for t in canon if t not in {'', 'geral'}]
    return set(canon)

def _similar(a: str, b: str, thresh: float = 0.75) -> bool:
    """Similaridade fuzzy simples."""
    if not a or not b:
        return False
    return SequenceMatcher(None, a, b).ratio() >= thresh

def match_area(areas_vaga, area_cand, use_fuzzy=True, thresh=0.75) -> str:
    """Regra: (1) macro igual -> sim; (2) interseção de termos -> sim; (3) fuzzy -> sim; senão 'nao'."""
    macro_vaga = _macro_area(areas_vaga)
    macro_cand = _macro_area(area_cand)

    if macro_vaga and macro_cand and (macro_vaga == macro_cand):
        return 'sim'

    set_vaga = _to_terms(areas_vaga)
    set_cand = _to_terms(area_cand)

    if set_vaga & set_cand:
        return 'sim'

    if use_fuzzy:
        for a in set_vaga:
            for b in set_cand:
                if _similar(a, b, thresh=thresh):
                    return 'sim'
    return 'nao'

df_final['match_area'] = df_final.apply(lambda r: match_area(r['areas_atuacao'], r['area_atuacao']), axis=1)


In [1474]:
#AVALIAR TERMOS SEMELHANTES NOS CAMPOS TÍTULO E OBJETIVO PROFISSIONAL (VAGA X CANDIDATO)

def _strip_accents(s: str) -> str:
    return unicodedata.normalize('NFKD', s).encode('ascii', 'ignore').decode('ascii')

# sinônimos
SINONIMOS = {
    r'\b(sr|senior|sênior|senior(a)?)\b': 'senior',
    r'\b(jr|junior|junior(a)?)\b': 'junior',
    r'\b(pl|pleno)\b': 'pleno',
    r'\bcoord\.?\b': 'coordenador',
    r'\bdev\.?\b': 'desenvolvedor',
    r'\bcons?ult(or|ora)?\b': 'consultor',
    r'\bpp\/qm\b': 'pp qm',
}

STOPWORDS = {'de','da','do','em','para','e','the','of','a','o','as','os','com'}

def _normalize(s):
    if pd.isna(s):
        return ''
    s = str(s).lower().strip()
    s = _strip_accents(s)
    s = re.sub(r'[-_]+', ' ', s)

    for patt, repl in SINONIMOS.items():
        s = re.sub(patt, repl, s)

    s = re.sub(r'[^a-z0-9/+\s]', ' ', s)

    s = re.sub(r'\s+', ' ', s).strip()
    return s

def _tokens(s):
    s = _normalize(s)

    parts = re.split(r'[\/\s]+', s)

    parts = [p for p in parts if p and p not in STOPWORDS]
    return parts

def _token_set(s):
    return set(_tokens(s))

def _seq_ratio(a, b):
    return SequenceMatcher(None, a, b).ratio()

def similares_titulo_objetivo(titulo, objetivo, thresh=0.85, jaccard_min=0.65) -> str:
    """Retorna 'sim' se titulo e objetivo forem similares, senão 'nao'."""
    if (pd.isna(titulo) or str(titulo).strip() == '') or (pd.isna(objetivo) or str(objetivo).strip() == ''):
        return 'nao'

    norm_t = _normalize(titulo)
    norm_o = _normalize(objetivo)

    if norm_t == norm_o:
        return 'sim'

    toks_t = _token_set(norm_t)
    toks_o = _token_set(norm_o)

    inter = toks_t & toks_o
    union = toks_t | toks_o
    if union:
        jacc = len(inter) / len(union)
        if jacc >= jaccard_min or (len(inter) >= max(2, min(len(toks_t), len(toks_o)) - 0)):
            return 'sim'

    sort_t = ' '.join(sorted(toks_t))
    sort_o = ' '.join(sorted(toks_o))
    if _seq_ratio(sort_t, sort_o) >= thresh:
        return 'sim'

    if norm_t in norm_o or norm_o in norm_t:
        return 'sim'

    return 'nao'

col_obj = 'objetivo_profissional' if 'objetivo_profissional' in df_final.columns else 'objetivo profissional'

df_final['match_titulo_objetivo'] = df_final.apply(
    lambda r: similares_titulo_objetivo(r['titulo'], r[col_obj], thresh=0.85, jaccard_min=0.65),
    axis=1
)



In [1475]:
#AVALIA SE O ESTADO E CIDADE DA VAGA SÃO IGUAIS AO ESTADO E CIDADE DO CANDIDATO
def normalize_text(text):
    if pd.isna(text) or text.strip() == '' or text.strip() == 'Não Informado':
        return 'Não Informado'
    text = text.lower().strip()
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('ascii')
    return text

df_final['estado_vaga_normalized'] = df_final['estado'].apply(normalize_text)
df_final['estado_candidato_normalized'] = df_final['estado_candidato'].apply(normalize_text)

df_final['match_estado'] = np.where(
    (df_final['estado_vaga_normalized'] != 'Não Informado') &
    (df_final['estado_candidato_normalized'] != 'Não Informado') &
    (df_final['estado_vaga_normalized'] == df_final['estado_candidato_normalized']),
    'Sim',
    'Não'
)


In [1476]:

df_final['cidade_vaga_normalized'] = df_final['cidade'].apply(normalize_text)
df_final['cidade_candidato_normalized'] = df_final['cidade_candidato'].apply(normalize_text)

df_final['match_cidade'] = np.where(
    (df_final['cidade_vaga_normalized'] != 'Não Informado') &
    (df_final['cidade_candidato_normalized'] != 'Não Informado') &
    (df_final['cidade_vaga_normalized'] == df_final['cidade_candidato_normalized']),
    'Sim',
    'Não'
)

display(df_final['match_cidade'].value_counts())

,count
match_cidade,
Não,35545
Sim,9526


In [1477]:
#AVALIA SE O NÍVEL DE HIERARQUIA PROFISSIONAL DO CANDIDATO É COMPATÍVEL (IGUAL OU MAIOR) AO NÍVEL EXIGIDO NA VAGA
nivel_mapping = {
    'Diretor': 10,
    'Gerente': 10,
    'Coordenador': 10,
    'Supervisor': 10,
    'Líder': 10,
    'Especialista': 9,
    'Sênior': 9,
    'Analista': 8,
    'Pleno': 8,
    'Trainee': 9,
    'Júnior': 8,
    'Assistente': 7,
    'Auxiliar': 7,
    'Técnico de Nível Médio': 7,
    'Aprendiz': 6,
    'Estagiário': 6,
    'Não Informado': 0
}

df_final['nivel_profissional_score'] = df_final['nivel profissional'].map(nivel_mapping).fillna(0)
df_final['nivel_profissional_candidato_score'] = df_final['nivel_profissional'].map(nivel_mapping).fillna(0)

In [1478]:
df_final['nivel_profissional_aderente'] = np.where(
    df_final['nivel_profissional_candidato_score'] >= df_final['nivel_profissional_score'],
    'Sim',
    'Não'
)



In [1479]:
#AVALIA SE O NÍVEL ACADÊMICO DO CANDIDATO É COMPATÍVEL (IGUAL OU MAIOR) AO NÍVEL EXIGIDO NA VAGA
nivel_academico_mapping = {
  'Doutorado Incompleto' : 9,
  'Doutorado Cursando' : 10,
  'Doutorado Completo' : 10,
  'Pós Graduação Completo' : 8,
  'Pós Graduação Incompleto' : 8,
  'Pós Graduação Cursando' : 8,
  'Mestrado Completo' : 9,
  'Mestrado Cursando' : 9,
  'Mestrado Incompleto' : 9,
  'Ensino Superior Completo' : 7,
  'Ensino Superior Cursando' : 7,
  'Ensino Superior Incompleto' : 6,
  'Ensino Técnico Completo' : 7,
  'Ensino Técnico Cursando' : 7,
  'Ensino Técnico Incompleto' : 7,
  'Ensino Fundamental Completo' : 5,
  'Ensino Fundamental Incompleto' : 5,
  'Ensino Médio Completo' : 6,
  'Ensino Médio Cursando' : 6,
  'Ensino Médio Incompleto' : 5,
  'Ensino Fundamental Cursando' : 5,
  'Não Informado': 0
}

df_final['nivel_academico_score'] = df_final['nivel_academico'].map(nivel_academico_mapping).fillna(0)
df_final['nivel_academico_candidato_score'] = df_final['nivel_academico_candidato'].map(nivel_academico_mapping).fillna(0)



In [1480]:
df_final['nivel_academico_aderente'] = np.where(
    df_final['nivel_academico_candidato_score'] >= df_final['nivel_academico_score'],
    'Sim',
    'Não'
)

In [1481]:
#AVALIA SE O NÍVEL DE IDIOMAS DO CANDIDATO É COMPATÍVEL (IGUAL OU MAIOR) AO NÍVEL EXIGIDO NA VAGA
nivel_idiomas_mapping = {
    'Avançado': 10,
    'Fluente': 10,
    'Intermediário': 8,
    'Técnico': 8,
    'Básico': 6,
    'Nenhum': 5,
    'Não Informado': 0
}

df_final['nivel_ingles_score'] = df_final['nivel_ingles'].map(nivel_academico_mapping).fillna(0)
df_final['nivel_ingles_candidato_score'] = df_final['nivel_ingles_candidato'].map(nivel_academico_mapping).fillna(0)
df_final['nivel_ingles_aderente'] = np.where(
    df_final['nivel_ingles_candidato_score'] >= df_final['nivel_ingles_score'],
    'Sim',
    'Não'
)


df_final['nivel_espanhol_score'] = df_final['nivel_espanhol'].map(nivel_academico_mapping).fillna(0)
df_final['nivel_espanhol_candidato_score'] = df_final['nivel_espanhol_candidato'].map(nivel_academico_mapping).fillna(0)
df_final['nivel_espanhol_aderente'] = np.where(
    df_final['nivel_espanhol_candidato_score'] >= df_final['nivel_espanhol_score'],
    'Sim',
    'Não'
)

In [1482]:
df_final.columns

Index(['id_vaga', 'titulo_vaga', 'vaga_sap', 'tipo_contratacao', 'pais',
       'estado', 'cidade', 'regiao', 'vaga_especifica_para_pcd',
       'nivel profissional', 'nivel_academico', 'nivel_ingles',
       'nivel_espanhol', 'areas_atuacao', 'principais_atividades',
       'competencia_tecnicas_e_comportamentais', 'demais_observacoes',
       'habilidades_comportamentais_necessarias', 'titulo', 'modalidade',
       'nome', 'codigo', 'situacao_candidado', 'codigo_profissional',
       'objetivo_profissional', 'pcd', 'titulo_profissional', 'area_atuacao',
       'conhecimentos_tecnicos', 'certificacoes', 'nivel_profissional',
       'nivel_academico_candidato', 'nivel_ingles_candidato',
       'nivel_espanhol_candidato', 'cursos', 'ano_conclusao', 'cargo_atual',
       'cidade_candidato', 'estado_candidato', 'remuneracao_num',
       'status_contratacao', 'faixa_remuneracao', 'possui_certificacao',
       'match_area', 'match_titulo_objetivo', 'estado_vaga_normalized',
       'estado_c

In [1483]:
df_final_modelagem = df_final[['id_vaga', 'vaga_sap', 'tipo_contratacao', 'vaga_especifica_para_pcd',
       'modalidade','nome', 'codigo_profissional', 'pcd',
       'faixa_remuneracao', 'possui_certificacao',
       'match_area', 'match_titulo_objetivo', 'match_estado', 'match_cidade',
       'nivel_profissional_aderente', 'nivel_academico_aderente',
       'nivel_ingles_aderente', 'nivel_espanhol_aderente', 'status_contratacao']]

In [1484]:
df_final_modelagem.to_csv('dados.csv', index=False)

In [1485]:
df_final.to_csv('dados_geral.csv', index=False)

### ETAPA 4 - MODELAGEM E AVALIAÇÃO DE RESULTADOS

In [1486]:
#pip install catboost

In [1487]:
import re
import numpy as np
import pandas as pd
from pathlib import Path
from typing import Optional, Tuple, Dict, List
from sklearn.model_selection import GroupKFold, StratifiedKFold
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import roc_auc_score, average_precision_score, brier_score_loss

In [1488]:
DATA_PATH   = Path("/content/dados.csv")
N_SPLITS    = 5
CAL_METHOD  = "sigmoid"
TOP_K       = 3
RANDOM_SEED = 42

In [1489]:
# Métricas de ranking por vaga
def hit_at_k(y_true: np.ndarray, y_score: np.ndarray, k: int) -> float:
    if len(y_true) == 0: return np.nan
    idx = np.argsort(-y_score)[:k]
    return float(y_true[idx].sum() > 0)

def map_at_k(y_true: np.ndarray, y_score: np.ndarray, k: int) -> float:
    if len(y_true) == 0: return np.nan
    idx = np.argsort(-y_score)[:k]
    y_sorted = y_true[idx]
    num_pos = y_true.sum()
    if num_pos == 0: return np.nan
    precisions, hits = [], 0
    for i, rel in enumerate(y_sorted, start=1):
        if rel == 1:
            hits += 1
            precisions.append(hits / i)
    return 0.0 if len(precisions) == 0 else float(np.sum(precisions) / min(k, int(num_pos)))

def ndcg_at_k(y_true: np.ndarray, y_score: np.ndarray, k: int) -> float:
    if len(y_true) == 0: return np.nan
    idx = np.argsort(-y_score)[:k]
    y_sorted = y_true[idx]
    dcg = float(np.sum(y_sorted / np.log2(np.arange(2, len(y_sorted) + 2))))
    ideal = np.sort(y_true)[::-1][:k]
    idcg = float(np.sum(ideal / np.log2(np.arange(2, len(ideal) + 2))))
    return np.nan if idcg == 0 else dcg / idcg

def ranking_metrics_by_group(df_scores: pd.DataFrame, grp: str, y_col: str, p_col: str, k: int) -> Dict[str, float]:
    mets = {"Hit@K": [], "MAP@K": [], "NDCG@K": []}
    for g, sub in df_scores.groupby(grp):
        yy = sub[y_col].to_numpy().astype(int)
        pp = sub[p_col].to_numpy().astype(float)
        mets["Hit@K"].append(hit_at_k(yy, pp, k))
        mets["MAP@K"].append(map_at_k(yy, pp, k))
        mets["NDCG@K"].append(ndcg_at_k(yy, pp, k))
    return {m: float(np.nanmean(v)) for m, v in mets.items()}

In [1490]:
# Carregamento
assert DATA_PATH.exists(), f"Arquivo não encontrado: {DATA_PATH}"
df = pd.read_csv(DATA_PATH)
print("Formato:", df.shape)
print("Algumas colunas:", list(df.columns)[:25])

Formato: (45071, 19)
Algumas colunas: ['id_vaga', 'vaga_sap', 'tipo_contratacao', 'vaga_especifica_para_pcd', 'modalidade', 'nome', 'codigo_profissional', 'pcd', 'faixa_remuneracao', 'possui_certificacao', 'match_area', 'match_titulo_objetivo', 'match_estado', 'match_cidade', 'nivel_profissional_aderente', 'nivel_academico_aderente', 'nivel_ingles_aderente', 'nivel_espanhol_aderente', 'status_contratacao']


In [1491]:
#  Alvo (y) = status_contratacao
def construir_target(series: pd.Series) -> np.ndarray:
    # Se for numérico/bool: >0 = 1, senão 0
    if pd.api.types.is_bool_dtype(series) or pd.api.types.is_numeric_dtype(series):
        return (pd.to_numeric(series, errors="coerce").fillna(0) > 0).astype(int).to_numpy()

    s = series.astype("string").str.lower().str.strip()
    # negativos explícitos
    neg = s.str.contains(r"(nao|não)\s*contrat|reprov|desclass|cancel|n[aã]o\s*aprov", regex=True, na=False)
    # positivos explícitos
    pos = s.str.contains(r"contrat|aprovad|hired|sim|yes", regex=True, na=False)
    y = (pos & ~neg).astype(int)
    return y.to_numpy()

assert "status_contratacao" in df.columns, "A coluna 'status_contratacao' não foi encontrada no CSV."
y = construir_target(df["status_contratacao"])
print(f"Alvo (status_contratacao): positivos={int(y.sum())}/{len(y)}")
assert y.sum()>0 and (len(y)-y.sum())>0, "Precisa de pelo menos 1 positivo e 1 negativo."


Alvo (status_contratacao): positivos=2609/45071


/tmp/ipython-input-4218001862.py:9: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  neg = s.str.contains(r"(nao|não)\s*contrat|reprov|desclass|cancel|n[aã]o\s*aprov", regex=True, na=False)


In [1492]:
#  Grupo por vaga
def detectar_grupo(_df: pd.DataFrame) -> Optional[str]:
    prefer = ["id_vaga","vaga_id","idvaga","id da vaga","id_da_vaga","ID_VAGA","ID VAGA"]
    for p in prefer:
        if p in _df.columns: return p

    for c in _df.columns:
        lc = c.lower().replace(" ", "").replace("_", "")
        if "vaga" in lc and "id" in lc:
            return c
    return None

grupo_col = detectar_grupo(df)
groups = df[grupo_col].to_numpy() if grupo_col is not None else None
print("Coluna de grupo (vaga):", grupo_col)

Coluna de grupo (vaga): id_vaga


In [1493]:
# Features
keys_out = [c for c in ["id_candidato","codigo_profissional","codigo","id_profissional","cpf","nome","email","telefone"] if c in df.columns]
if grupo_col and grupo_col not in keys_out:
    keys_out.append(grupo_col)

In [1494]:
# Base X bruta (remove alvo e chaves)
base_drop = [c for c in ["status_contratacao"] + keys_out if c in df.columns]
X_base = df.drop(columns=base_drop, errors="ignore").copy()

In [1495]:
# Regressão Logística
X_lr = X_base.copy()
n = len(df)
obj_cols = X_lr.select_dtypes(include=["object","category","bool"]).columns.tolist()
long_text_lr = [c for c in obj_cols if X_lr[c].astype("string").str.len().fillna(0).mean() > 50]
high_card_lr  = [c for c in obj_cols if X_lr[c].nunique(dropna=True) > 0.30*n]
drop_lr = sorted(set(long_text_lr + high_card_lr))
if drop_lr:
    print("LR - Removendo colunas pesadas:", drop_lr[:15], "..." if len(drop_lr)>15 else "")
    X_lr = X_lr.drop(columns=drop_lr, errors="ignore")

num_cols_lr = X_lr.select_dtypes(include=["number","bool","float","int"]).columns.tolist()
cat_cols_lr = X_lr.select_dtypes(include=["object","category"]).columns.tolist()

pre_lr = ColumnTransformer(
    transformers=[
        ("num", SimpleImputer(strategy="median"), num_cols_lr),
        ("cat", Pipeline([
            ("imp", SimpleImputer(strategy="most_frequent")),
            ("ohe", OneHotEncoder(handle_unknown="ignore"))
        ]), cat_cols_lr),
    ],
    remainder="drop"
)
clf_lr = LogisticRegression(max_iter=300, class_weight="balanced", solver="saga", random_state=RANDOM_SEED, n_jobs=1)


In [1496]:
# Funções de CV com calibração por fold
def oof_lr_calibrado(X: pd.DataFrame, y: np.ndarray, groups: Optional[np.ndarray], n_splits: int, method: str) -> Tuple[np.ndarray, np.ndarray]:
    if groups is not None:
        splitter = GroupKFold(n_splits=n_splits)
        splits = splitter.split(X, y, groups)
    else:
        splitter = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_SEED)
        splits = splitter.split(X, y)

    oof = np.zeros(len(X), dtype=float)
    fold_ids = np.zeros(len(X), dtype=int)

    for fold, (tr, va) in enumerate(splits, start=1):
        X_tr, X_va = X.iloc[tr], X.iloc[va]
        y_tr, y_va = y[tr], y[va]

        pipe = Pipeline([("pre", pre_lr), ("clf", clf_lr)])
        pipe.fit(X_tr, y_tr)

        calibrator = CalibratedClassifierCV(pipe, cv="prefit", method=method)
        calibrator.fit(X_va, y_va)

        proba_va = calibrator.predict_proba(X_va)[:, 1]
        oof[va] = proba_va
        fold_ids[va] = fold

        print(f"[LR Fold {fold}] n_va={len(va)} | ROC_AUC={roc_auc_score(y_va, proba_va):.4f} | PR_AUC={average_precision_score(y_va, proba_va):.4f}")
    return oof, fold_ids


In [1497]:
# Rodar CV (OOF)
print("\n>> Regressão Logística (OOF calibrado)")
probs_oof_lr, fold_lr = oof_lr_calibrado(X_lr, y, groups, N_SPLITS, CAL_METHOD)


>> Regressão Logística (OOF calibrado)


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(


[LR Fold 1] n_va=9015 | ROC_AUC=0.7934 | PR_AUC=0.2898


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(


[LR Fold 2] n_va=9014 | ROC_AUC=0.8010 | PR_AUC=0.3056


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(


[LR Fold 3] n_va=9014 | ROC_AUC=0.7979 | PR_AUC=0.2710


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(


[LR Fold 4] n_va=9014 | ROC_AUC=0.7994 | PR_AUC=0.3272
[LR Fold 5] n_va=9014 | ROC_AUC=0.7846 | PR_AUC=0.2930


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(


In [1498]:
# Métricas globais
roc_lr, pr_lr, brier_lr = roc_auc_score(y, probs_oof_lr), average_precision_score(y, probs_oof_lr), brier_score_loss(y, probs_oof_lr)

df_compare = pd.DataFrame({
    "modelo": ["LogisticRegression"],
    "ROC_AUC_OOF": [roc_lr],
    "PR_AUC_OOF": [pr_lr],
    "Brier_OOF": [brier_lr],
})
print("\nComparativo (OOF calibrado)")
print(df_compare)


Comparativo (OOF calibrado)
               modelo  ROC_AUC_OOF  PR_AUC_OOF  Brier_OOF
0  LogisticRegression         0.80        0.31       0.05


In [1499]:
# Ranking por vaga (OOF)
cols_keep = keys_out.copy()
df_scores = df[cols_keep].copy() if cols_keep else pd.DataFrame(index=df.index)
df_scores["y"] = y
df_scores["prob_oof_calibrada_lr"] = probs_oof_lr

if grupo_col:
    m_rank_lr = ranking_metrics_by_group(
        df_scores.sort_values([grupo_col, "prob_oof_calibrada_lr"], ascending=[True, False]),
        grp=grupo_col, y_col="y", p_col="prob_oof_calibrada_lr", k=TOP_K
    )
    df_rank = pd.DataFrame([
        {"modelo":"LogisticRegression", **{f"{k}@{TOP_K}":v for k,v in m_rank_lr.items()}},
    ])
    print(f"\n Ranking por vaga (OOF calibrado, K={TOP_K})")
    print(df_rank)
else:
    df_rank = pd.DataFrame()


 Ranking por vaga (OOF calibrado, K=3)
               modelo  Hit@K@3  MAP@K@3  NDCG@K@3
0  LogisticRegression     0.17     0.71      0.74


In [1500]:
# Modelo final calibrado (produção)
# LR final
pipe_lr = Pipeline([("pre", pre_lr), ("clf", clf_lr)])
final_lr = CalibratedClassifierCV(pipe_lr, cv=3, method=CAL_METHOD)
final_lr.fit(X_lr, y)
probs_full_lr = final_lr.predict_proba(X_lr)[:, 1]

df_scores["prob_full_calibrada_lr"] = probs_full_lr


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [1501]:
#  TOP-3 por vaga com o modelo final
if grupo_col:
    df_topk_lr = (
        df_scores.sort_values([grupo_col, "prob_full_calibrada_lr"], ascending=[True, False])
                 .groupby(grupo_col, as_index=False).head(TOP_K).reset_index(drop=True)
    )

else:
    df_topk_lr = df_scores.sort_values("prob_full_calibrada_lr", ascending=False).head(TOP_K).reset_index(drop=True)


In [1502]:
# Exibir amostras
print("\n df_scores (amostra)")
print(df_scores.head(10))

if grupo_col:
    print(f"\n TOP-{TOP_K} por vaga — LR (amostra)")
    print(df_topk_lr.head(10))


# Exportar CSV
# df_scores.to_csv("/mnt/data/probabilidades_comparativo.csv", index=False, encoding="utf-8-sig")
# df_topk_lr.to_csv("/mnt/data/topk_por_vaga_lr.csv", index=False, encoding="utf-8-sig")


 df_scores (amostra)
   codigo_profissional                nome  id_vaga  y  prob_oof_calibrada_lr  \
0                11010    Dante Nascimento     5185  0                   0.08   
1                26770        Samuel Costa     5184  0                   0.11   
2                26759   Maria Laura Brito     5184  0                   0.11   
3                26758       Raul Monteiro     5184  0                   0.11   
4                26757   José Miguel Cunha     5184  0                   0.11   
5                26756      Gael das Neves     5184  0                   0.11   
6                26722  Esther Casa Grande     5184  0                   0.11   
7                26719         Léo Peixoto     5184  0                   0.11   
8                26570       Isabella Rios     5184  0                   0.11   
9                26139         Matteo Lima     5184  0                   0.11   

   prob_full_calibrada_lr  
0                    0.07  
1                    0.10  
2 

In [1503]:
df_scores

,codigo_profissional,nome,id_vaga,y,prob_oof_calibrada_lr,prob_full_calibrada_lr
0,11010,Dante Nascimento,5185,0,0.08,0.07
1,26770,Samuel Costa,5184,0,0.11,0.10
2,26759,Maria Laura Brito,5184,0,0.11,0.10
3,26758,Raul Monteiro,5184,0,0.11,0.10
4,26757,José Miguel Cunha,5184,0,0.11,0.10
...,...,...,...,...,...,...
45066,46361,Mateus Cavalcante,12364,0,0.05,0.02
45067,46319,Marina Caldeira,12364,0,0.01,0.01
45068,46139,Benjamin Andrade,12364,0,0.02,0.01
45069,29568,Olívia Silva,12364,0,0.02,0.02


In [1504]:
df_topk_lr

,codigo_profissional,nome,id_vaga,y,prob_oof_calibrada_lr,prob_full_calibrada_lr
0,12585,Luiz Felipe Costela,2,1,0.66,0.73
1,12598,Sr. Luiz Fernando Fernandes,3,0,0.04,0.06
2,12595,Dra. Allana Pacheco,3,0,0.04,0.06
3,12618,Dra. Kamilly Nascimento,4,0,0.04,0.06
4,12626,Luiz Felipe Moraes,5,0,0.07,0.06
...,...,...,...,...,...,...
23670,15271,Lorena da Cunha,14217,0,0.01,0.04
23671,30750,Sra. Eloah Correia,14217,0,0.01,0.04
23672,40384,Dr. Vitor Hugo Silva,14218,0,0.04,0.02
23673,16828,Ana Cardoso,14220,0,0.02,0.03


In [1505]:
#TESTE
df_topk_lr[df_topk_lr['id_vaga'] == 4531]

,codigo_profissional,nome,id_vaga,y,prob_oof_calibrada_lr,prob_full_calibrada_lr
7808,25360,Alexia Barbosa,4531,0,0.04,0.03
7809,25364,Sra. Yasmin Fernandes,4531,1,0.02,0.01


In [1506]:
#TESTE
df_scores[df_scores['id_vaga'] == 4531]

,codigo_profissional,nome,id_vaga,y,prob_oof_calibrada_lr,prob_full_calibrada_lr
33039,25364,Sra. Yasmin Fernandes,4531,1,0.02,0.01
33040,25360,Alexia Barbosa,4531,0,0.04,0.03


In [1507]:
#TESTE
df_scores_sorted = df_scores.sort_values(['id_vaga', 'prob_oof_calibrada_lr'], ascending=[True, False])

In [1508]:
#TESTE
df_scores_sorted[df_scores_sorted['id_vaga'] == 4581]

,codigo_profissional,nome,id_vaga,y,prob_oof_calibrada_lr,prob_full_calibrada_lr
32908,25356,Dr. Gustavo Henrique Viana,4581,0,0.03,0.01
32909,25318,Isabelly Duarte,4581,0,0.03,0.02
32906,25526,Leandro Cunha,4581,0,0.03,0.02
32907,25515,Sr. Eduardo Abreu,4581,0,0.01,0.02


### ETAPA 5 - PIPELINE DO MODELO

In [1509]:

# Pipeline Regressão Logística

import re
import numpy as np
import pandas as pd
from pathlib import Path

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibratedClassifierCV
from joblib import dump


DATA_PATH   = Path("/content/dados.csv")
PIPE_PATH   = Path("pipeline_logistica.joblib")          # pipeline crua (pre + LR)
CALIB_PATH  = Path("modelo_logistica_calibrado.joblib")  # pipeline calibrada
CAL_METHOD  = "sigmoid"
RANDOM_SEED = 42


assert DATA_PATH.exists(), f"Arquivo não encontrado: {DATA_PATH}"
df = pd.read_csv(DATA_PATH)

def construir_target(series: pd.Series) -> np.ndarray:
    if pd.api.types.is_bool_dtype(series) or pd.api.types.is_numeric_dtype(series):
        return (pd.to_numeric(series, errors="coerce").fillna(0) > 0).astype(int).to_numpy()
    s = series.astype("string").str.lower().str.strip()
    neg = s.str.contains(r"(nao|não)\s*contrat|reprov|desclass|cancel|n[aã]o\s*aprov", regex=True, na=False)
    pos = s.str_contains = s.str.contains(r"contrat|aprovad|hired|sim|yes", regex=True, na=False)
    y = (pos & ~neg).astype(int)
    return y.to_numpy()

assert "status_contratacao" in df.columns, "Faltou a coluna 'status_contratacao'."
y = construir_target(df["status_contratacao"])
assert y.sum() > 0 and (len(y) - y.sum()) > 0, "Precisa de pelo menos 1 positivo e 1 negativo."


keys_out = [c for c in ["id_candidato","codigo_profissional","codigo","id_profissional","cpf","nome","email","telefone","id_vaga","vaga_id","idvaga","id_da_vaga","ID_VAGA","ID VAGA"] if c in df.columns]
X = df.drop(columns=[c for c in ["status_contratacao"] + keys_out if c in df.columns], errors="ignore").copy()

n = len(df)
obj_cols = X.select_dtypes(include=["object","category","bool"]).columns.tolist()
long_text_lr = [c for c in obj_cols if X[c].astype("string").str.len().fillna(0).mean() > 50]
high_card_lr  = [c for c in obj_cols if X[c].nunique(dropna=True) > 0.30 * n]
drop_lr = sorted(set(long_text_lr + high_card_lr))
if drop_lr:
    print("LR - Removendo colunas pesadas:", drop_lr[:15], "..." if len(drop_lr) > 15 else "")
    X = X.drop(columns=drop_lr, errors="ignore")

num_cols = X.select_dtypes(include=["number","bool","float","int"]).columns.tolist()
cat_cols = X.select_dtypes(include=["object","category"]).columns.tolist()

pre_lr = ColumnTransformer(
    transformers=[
        ("num", SimpleImputer(strategy="median"), num_cols),
        ("cat", Pipeline([
            ("imp", SimpleImputer(strategy="most_frequent")),
            ("ohe", OneHotEncoder(handle_unknown="ignore"))
        ]), cat_cols),
    ],
    remainder="drop"
)

clf_lr = LogisticRegression(
    max_iter=300,
    class_weight="balanced",
    solver="saga",
    random_state=RANDOM_SEED,
)

pipe_lr = Pipeline([
    ("pre", pre_lr),
    ("clf", clf_lr),
])

# 7) Treina pipeline crua
pipe_lr.fit(X, y)

# 8) Calibração das probabilidades
final_lr = CalibratedClassifierCV(pipe_lr, cv=3, method=CAL_METHOD)
final_lr.fit(X, y)

# 9) Salva modelos
dump(pipe_lr, PIPE_PATH)
dump(final_lr, CALIB_PATH)
print(f"Pipeline salva em: {PIPE_PATH}")
print(f"Modelo calibrado salvo em: {CALIB_PATH}")


/tmp/ipython-input-3750878137.py:31: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  neg = s.str.contains(r"(nao|não)\s*contrat|reprov|desclass|cancel|n[aã]o\s*aprov", regex=True, na=False)
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Pipeline salva em: pipeline_logistica.joblib
Modelo calibrado salvo em: modelo_logistica_calibrado.joblib


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
